In [1]:
!pip install ultralytics 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.5/983.5 kB 14.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found

In [2]:
from ultralytics import YOLO
import cv2

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
# Constants
FOCAL_LENGTH_PX = 3600  # iPhone 12 Pro Max
class_heights_m = {
    '80': 0.80,
    '120': 1.20,
    '140': 1.40,
    '160': 1.60,
    '200': 2.00,
    '210': 2.10,
    '220': 2.20,
    '380': 3.80,
    '650': 8.50
}

# Load model
model = YOLO('/kaggle/input/modelytorun/pytorch/default/1/best (2).pt')

# Load image
img_path = '/kaggle/input/the-dataset-to-send/test/images/IEEE-Office_img15_jpg.rf.e80a7210e8cddd4ec3479579304d220a.jpg'
image = cv2.imread(img_path)

# Inference
results = model(img_path)

# Collect distances
distances = []

# Process detections
for result in results:
    boxes = result.boxes
    for i in range(len(boxes)):
        cls_id = int(boxes.cls[i])
        class_name = str(model.names[cls_id])  # ensure it's a string like '120'

        xyxy = boxes.xyxy[i].cpu().numpy()
        height_px = int(xyxy[3] - xyxy[1])

        real_height_m = class_heights_m.get(class_name)
        if real_height_m is not None and height_px > 0:
            distance_m = (FOCAL_LENGTH_PX * real_height_m) / height_px
            distances.append(distance_m)
            print(f"Window Type: {class_name}, Height: {height_px}px, Distance: {distance_m:.2f} m")
        else:
            print(f"Unknown class or invalid bounding box height: {class_name}")

# Compute and display average distance
if distances:
    avg_distance = sum(distances) / len(distances)
    print(f"\n📏 Average Estimated Distance: {avg_distance:.2f} meters")
else:
    print("\n⚠️ No valid detections for distance calculation.")



image 1/1 /kaggle/input/the-dataset-to-send/test/images/IEEE-Office_img15_jpg.rf.e80a7210e8cddd4ec3479579304d220a.jpg: 640x640 2 120s, 640.2ms
Speed: 16.9ms preprocess, 640.2ms inference, 25.0ms postprocess per image at shape (1, 3, 640, 640)
Window Type: 120, Height: 32px, Distance: 135.00 m
Window Type: 120, Height: 41px, Distance: 105.37 m

📏 Average Estimated Distance: 120.18 meters
